In [653]:
from datetime import datetime
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from dateutil.parser import parse

ProgressBar().register()

In [24]:
FHV = dd.read_csv('May_to_Aug_FHV_Data.csv')
FHV = FHV.drop(columns='Unnamed: 0').reset_index(drop=True)

In [ ]:
start = dd.to_datetime('2020-05-06 00:00:00')

In [33]:
FHV.PUTime = dd.to_datetime(FHV.PUTime)
FHV.DOTime = dd.to_datetime(FHV.DOTime)

In [66]:
FHV['Hour'] = FHV.PUTime.dt.hour
FHV['DOW'] = FHV.PUTime.dt.dayofweek

In [43]:
FHV['overnight'] = 0

In [53]:
FHV['overnight'] = FHV['overnight'].mask(((FHV['Hour'] > 0) & (FHV['Hour'] < 6)), 1)

In [654]:
FHV[FHV['overnight']==1]['Duration'].mean().compute()

[########################################] | 100% Completed |  2min  5.5s
[########################################] | 100% Completed |  2min  5.5s


23.071644221978115

In [655]:
FHV = FHV[FHV['Duration'] >= 5].compute()

[########################################] | 100% Completed |  2min 11.6s
[########################################] | 100% Completed |  2min 11.7s


In [686]:
for c in FHV['Company'].unique():
    print(c,'total ride percentage:', len(FHV[FHV['Company']==c]) / len(FHV))
    print(c,'overnight percentage:', len(FHV[(FHV['overnight']==1) & (FHV['Company']==c)]) / len(FHV[(FHV['overnight']==1)]))
    print('\n')


Uber total ride percentage: 0.6533638592230323
Uber overnight percentage: 0.6322475057073821


Lyft total ride percentage: 0.3238106646208832
Lyft overnight percentage: 0.3519408963512366


Via total ride percentage: 0.02282547615608455
Via overnight percentage: 0.015811597941381345




In [689]:
FHV[FHV['overnight']==1].groupby(['PULocationID','DOLocationID'])['Company'].count().sort_values(ascending=False)

PULocationID  DOLocationID
61            76              1810
76            61              1680
225           76              1198
42            265             1154
39            61              1125
                              ... 
171           47                 1
              62                 1
49            98                 1
              96                 1
1             42                 1
Name: Company, Length: 48169, dtype: int64

In [159]:
non_overnight_pickups = FHV[FHV['overnight']==0].PULocationID.value_counts().compute()

In [160]:
overnight_pickups = FHV[FHV['overnight']==1].PULocationID.value_counts().compute()

In [84]:
taxi_zones = gpd.read_file('./taxi_zones__7_/taxi_zones.shp')

In [89]:
ride_pct_change = ((FHV[FHV['overnight']==1].PULocationID.value_counts() \
                    - FHV[FHV['overnight']==0].PULocationID.value_counts())/\
                   FHV[FHV['overnight']==0].PULocationID.value_counts()).compute()

In [118]:
pd.Series(ride_pct_change.sort_values(ascending=False)).iloc[:20]

207   -0.529016
30    -0.805679
126   -0.848193
2     -0.861635
57    -0.862991
37    -0.867701
99    -0.868657
36    -0.870975
58    -0.871403
120   -0.871784
173   -0.872243
77    -0.873824
10    -0.875806
152   -0.876772
150   -0.876915
177   -0.877290
147   -0.879774
235   -0.879969
169   -0.880068
78    -0.880545
Name: PULocationID, dtype: float64

### Merging ACS and subway ridership data with FHV Data

In [701]:
# Import pct_change in subway ridership from May through August between 2019 and 2020

subway = pd.read_csv('MLC_data-main 3/data/ridership_by_taxizone/ridership_by_taxizone.csv')
subway = subway.drop(columns=['Unnamed: 0', '2019_entries', '2019_exits','2020_entries','2020_exits'])
subway['mean_pct_change'] = (subway['entries__pct_change'] + subway['exits_pct_change']) / 2

In [702]:
subway = subway.drop(columns=['entries__pct_change','exits_pct_change'])

In [703]:
# Import 2019 ACS variables for census_tracts and taxi

acs = pd.read_csv('MLC_data-main 3/data/census_zones/census_zones.csv')

In [704]:
# select relevant variables from ACS table

acs_var_names = {'healthcare':'SE_B17008_004','food_service':'SE_B17008_006','maintenance':'SE_B17008_007',\
                 'retail_sales':'SE_B17008_009', 'pct_white':'SE_A03001_002', 'hh_median_income':'SE_A14006_001',\
                 'pct_transit_commute':'SE_A09005_003','transit_12_5':'ACS19_5yr_B08132047','transit_60min+':'ACS19_5yr_B08134070'}

acs_var_names = {v: k for k, v in acs_var_names.items()}

acs_vars = [s.strip() for s in list(acs_var_names.keys())]

In [705]:
pd.DataFrame.from_dict(acs_var_names, orient='index')

,0
SE_B17008_004,healthcare
SE_B17008_006,food_service
SE_B17008_007,maintenance
SE_B17008_009,retail_sales
SE_A03001_002,pct_white
SE_A14006_001,hh_median_income
SE_A09005_003,pct_transit_commute
ACS19_5yr_B08132047,transit_12_5
ACS19_5yr_B08134070,transit_60min+


In [570]:
zone_acs1 = acs[['LocationID'] + acs_vars].groupby('LocationID').sum().drop(columns='SE_A14006_001')
zone_acs2 = acs[['LocationID','SE_A14006_001']].groupby('LocationID').mean()
zone_acs = zone_acs1.merge(zone_acs2, on='LocationID')

In [571]:
zone_acs.head()

,SE_B17008_004,SE_B17008_006,SE_B17008_007,SE_B17008_009,SE_A03001_002,SE_A09005_003,ACS19_5yr_B08132047,ACS19_5yr_B08134070,SE_A14006_001
LocationID,,,,,,,,,
2.0,0,0,0,0,0,0,0,0,NaN
3.0,794,531,697,1183,9220,5752,198,3922,78678.545455
4.0,643,512,394,1357,10121,7346,158,1255,48225.166667
5.0,361,290,258,1455,24498,2602,28,2322,91298.500000
6.0,334,409,182,565,11378,2640,36,1767,78611.000000


In [572]:
zone_acs.columns = [acs_var_names[x] for x in zone_acs.columns]

In [573]:
zone_acs['essential_workers'] = zone_acs.iloc[:,:4].sum(axis=1)

In [574]:
zone_acs = zone_acs.reset_index().drop(columns=['healthcare','food_service','maintenance','retail_sales'])

In [191]:
#essential_by_zone = pd.DataFrame(acs.groupby('LocationID')['SE_A17004_011'].mean())
#essential_by_zone = essential_by_zone.rename(columns={'SE_A17004_011':'essential_count'}).reset_index()
#essential_by_zone.head()

In [708]:
#merge FHV_test data with ACS and subway variables by Taxi Zone

FHV_merged = FHV.merge(zone_acs, left_on='PULocationID', right_on='LocationID', how='left').drop(columns='LocationID')\
    .merge(zone_acs, left_on='DOLocationID', right_on='LocationID', how='left').drop(columns='LocationID')

FHV_merged = FHV_merged.merge(subway, left_on='PULocationID', right_on='LocationID', how='left').drop(columns='LocationID')\
    .merge(subway, left_on='DOLocationID', right_on='LocationID', how='left').dropna()



In [709]:
FHV_merged.head()

,Company,PULocationID,DOLocationID,PUTime,DOTime,Duration,Hour,overnight,DOW,pct_white_x,...,essential_workers_x,pct_white_y,pct_transit_commute_y,transit_12_5_y,transit_60min+_y,hh_median_income_y,essential_workers_y,mean_pct_change_x,LocationID,mean_pct_change_y
0,Uber,74,32,2020-05-06 00:43:42,2020-05-06 00:59:27,15.750000,0,0,2,11740.0,...,7489.0,11322.0,10153.0,500.0,5815.0,40033.125000,4909.0,-0.633493,32.0,-0.540503
1,Lyft,48,42,2020-05-06 00:18:53,2020-05-06 00:34:48,15.916667,0,0,2,22815.0,...,3878.0,13339.0,28784.0,848.0,4921.0,52185.187500,9446.0,-0.782626,42.0,-0.631897
2,Uber,80,220,2020-05-06 00:14:54,2020-05-06 00:41:32,26.633333,0,0,2,20974.0,...,4901.0,18221.0,8040.0,108.0,4939.0,73823.111111,2977.0,-0.616585,220.0,-0.587958
3,Via,136,141,2020-05-06 00:03:09,2020-05-06 00:20:31,17.366667,0,0,2,5840.0,...,5918.0,34206.0,13259.0,0.0,799.0,125009.750000,3842.0,-0.550806,141.0,-0.791837
4,Via,163,226,2020-05-06 00:56:35,2020-05-06 01:11:52,15.283333,0,0,2,5744.0,...,641.0,29173.0,19267.0,280.0,3295.0,61967.142857,6996.0,-0.780825,226.0,-0.658266


In [710]:
FHV_merged = FHV_merged.rename(columns={'essential_count_x':'PU_essential_count','essential_count_y':'DO_essential_count',\
                          'mean_pct_change_x':'PU_subway_change','mean_pct_change_y':'DO_subway_change'}).drop(columns='LocationID')



### Exploratory Data Analysis


In [ ]:
FHV_merged[FHV_merged['overnight']==1].mean()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


In [597]:
FHV_merged[(FHV_merged['PULocationID']==76) & (FHV_merged['DOLocationID']==76)]

,PULocationID,DOLocationID,PUTime,DOTime,Duration,overnight,pct_white_x,pct_transit_commute_x,transit_12_5_x,transit_60min+_x,...,0,1,2,3,4,5,6,Lyft,Uber,Via
252,76,76,2020-05-06 04:25:59,2020-05-06 04:30:25,4.433333,1,15275.0,24706.0,970.0,9723.0,...,0,0,1,0,0,0,0,0,1,0
280,76,76,2020-05-06 04:28:10,2020-05-06 04:32:17,4.116667,1,15275.0,24706.0,970.0,9723.0,...,0,0,1,0,0,0,0,0,1,0
281,76,76,2020-05-06 04:38:39,2020-05-06 04:44:37,5.966667,1,15275.0,24706.0,970.0,9723.0,...,0,0,1,0,0,0,0,0,1,0
473,76,76,2020-05-06 04:50:55,2020-05-06 05:01:09,10.233333,1,15275.0,24706.0,970.0,9723.0,...,0,0,1,0,0,0,0,0,1,0
753,76,76,2020-05-06 04:42:25,2020-05-06 04:48:48,6.383333,1,15275.0,24706.0,970.0,9723.0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790945,76,76,2020-08-31 06:09:23,2020-08-31 06:12:31,3.133333,0,15275.0,24706.0,970.0,9723.0,...,1,0,0,0,0,0,0,0,1,0
1791071,76,76,2020-08-31 06:42:18,2020-08-31 06:47:51,5.550000,0,15275.0,24706.0,970.0,9723.0,...,1,0,0,0,0,0,0,1,0,0
1791101,76,76,2020-08-31 06:28:23,2020-08-31 06:34:16,5.883333,0,15275.0,24706.0,970.0,9723.0,...,1,0,0,0,0,0,0,1,0,0
1791171,76,76,2020-08-31 06:43:57,2020-08-31 06:49:36,5.650000,0,15275.0,24706.0,970.0,9723.0,...,1,0,0,0,0,0,0,1,0,0


### Classification Tests

In [647]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [585]:
FHV_merged = FHV_merged.join(pd.get_dummies(FHV_merged['DOW'])).drop(columns=['DOW','Hour'])

In [586]:
FHV_merged = FHV_merged.join(pd.get_dummies(FHV_merged['Company'])).drop(columns='Company')

In [587]:
FHV_merged.drop(columns=['PUTime','DOTime','overnight'])

,PULocationID,DOLocationID,Duration,pct_white_x,pct_transit_commute_x,transit_12_5_x,transit_60min+_x,hh_median_income_x,essential_workers_x,pct_white_y,...,0,1,2,3,4,5,6,Lyft,Uber,Via
0,260,129,10.700000,18985.0,15002.0,385.0,2996.0,64765.833333,6166.0,46445.0,...,0,0,1,0,0,0,0,0,1,0
1,146,74,18.816667,2927.0,3140.0,68.0,489.0,73450.000000,1179.0,11740.0,...,0,0,1,0,0,0,0,0,1,0
2,74,42,4.200000,11740.0,17396.0,311.0,2752.0,33484.090909,7489.0,13339.0,...,0,0,1,0,0,0,0,0,1,0
3,42,235,11.933333,13339.0,28784.0,848.0,4921.0,52185.187500,9446.0,5382.0,...,0,0,1,0,0,0,0,0,1,0
4,37,17,8.200000,25825.0,25923.0,814.0,7703.0,59154.333333,10271.0,35888.0,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791813,75,237,7.900000,23479.0,16858.0,367.0,2400.0,50623.818182,5960.0,15803.0,...,1,0,0,0,0,0,0,0,1,0
1791814,236,113,15.166667,32246.0,11968.0,27.0,599.0,177116.125000,2915.0,16269.0,...,1,0,0,0,0,0,0,0,1,0
1791815,69,137,28.183333,8405.0,17493.0,906.0,6858.0,33030.307692,10351.0,15913.0,...,1,0,0,0,0,0,0,0,1,0
1791816,181,140,21.450000,52345.0,28187.0,179.0,4473.0,142617.000000,3852.0,22657.0,...,1,0,0,0,0,0,0,0,1,0


In [588]:
X = FHV_merged.drop(columns=['PUTime','DOTime','overnight'])
y = FHV_merged['overnight']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=9,test_size=0.4)

In [589]:
lm = LogisticRegression()
lm.fit(X_train,y_train)
lm.score(X_train,y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5605901278223819

In [590]:
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
print('IS R2:',dt.score(X_train,y_train))
print('OS R2:',dt.score(X_test,y_test))

IS R2: 0.9918112518477725
OS R2: 0.606187295332138


In [623]:
rf = RandomForestClassifier(n_estimators=30)
rf.fit(X_train,y_train)
print('IS R2:',rf.score(X_train,y_train))
print('OS R2:',rf.score(X_test,y_test))

IS R2: 0.9867586446769658
OS R2: 0.6318039292751345


In [630]:
param_grid = {'max_depth':range(2,13)}
rf = RandomForestClassifier(n_estimators=30)
gs = GridSearchCV(rf,param_grid=param_grid,scoring='accuracy')
rs = gs.fit(X_train,y_train)
pred=rs.predict(X_test)
print(rs.best_params_)
print(roc_auc_score(np.array(y_test),pred))

{'max_depth': 12}


NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [640]:
pred=rs.predict(X_train)

In [641]:
correct=1.0*(pred==y_train).sum()/len(y_train)
correct

0.6463163386568506

In [648]:
accuracy_score(y_train,pred)

0.6463163386568506

In [633]:
print(roc_auc_score(np.array(y_test),pred))

0.6856680241632485


In [394]:
FHV_test.groupby(['PULocationID','DOLocationID']).agg({'Company':'count', 'overnight':'sum'}).sort_values('overnight',ascending=False)

Company  overnight
PULocationID DOLocationID                    
76           76               6587       3377
39           39               4784       2411
61           61               2827       1675
215          130              3004       1663
205          130              2762       1463
...                            ...        ...
87           52                  1          0
218          133                 3          0
             126                 1          0
87           84                  1          0
             34                  7          0

[44057 rows x 2 columns]